### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.14.0'

### Enable Eager Execution if you are using tensorflow 1.x

In [3]:
#Enable Eager Execution
tf.enable_eager_execution()

### Collect Data

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [8]:
data = data.drop(columns=['date', 'symbol'], axis=1)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [9]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [10]:
data = data.head(1000)

### Convert Float64 to Float32

In [11]:
data.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [12]:
data = data.astype('float32')

In [13]:
data.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

### Divide the data into train and test sets

In [16]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['close'], axis=1)
y = data.close

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=7)


(1000, 4) (1000,)


### Normalize Train and Test Data 

In [35]:
from sklearn.preprocessing import Normalizer
import numpy as np

transformer = Normalizer()

X_train = transformer.fit_transform(X_train)
y_train = np.array(y_train) # This is importanat

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(700, 4) (700,)
(300, 4) (300,)


## Building the graph in tensorflow

2.Define Weights and Bias

In [36]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [37]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [38]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [39]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b


## Execute the Graph for 100 epochs and observe the loss

In [40]:
#Train for 100 Steps
for i in range(100):
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 8002.6514
Current Loss on iteration 1 7663.568
Current Loss on iteration 2 7351.0693
Current Loss on iteration 3 7063.065
Current Loss on iteration 4 6797.6504
Current Loss on iteration 5 6553.033
Current Loss on iteration 6 6327.599
Current Loss on iteration 7 6119.838
Current Loss on iteration 8 5928.3643
Current Loss on iteration 9 5751.9062
Current Loss on iteration 10 5589.279
Current Loss on iteration 11 5439.4014
Current Loss on iteration 12 5301.2764
Current Loss on iteration 13 5173.977
Current Loss on iteration 14 5056.661
Current Loss on iteration 15 4948.547
Current Loss on iteration 16 4848.9023
Current Loss on iteration 17 4757.0684
Current Loss on iteration 18 4672.4365
Current Loss on iteration 19 4594.4424
Current Loss on iteration 20 4522.562
Current Loss on iteration 21 4456.313
Current Loss on iteration 22 4395.265
Current Loss on iteration 23 4338.999
Current Loss on iteration 24 4287.1455
Current Loss on iteration 25 4239.3525
Current L

### Get the shapes and values of W and b

In [41]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5353245e-03]
 [2.5108384e-03]
 [2.5564781e-03]
 [3.3674854e+01]]
Bias:
 [33.67486]


In [42]:
X_train[0]

array([1.3583756e-05, 1.3455608e-05, 1.3683873e-05, 1.0000000e+00],
      dtype=float32)

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [43]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [44]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 328us/sample - loss: 5947.5628
Epoch 2/100
700/700 [==============================] - 0s 78us/sample - loss: 4070.1454
Epoch 3/100
700/700 [==============================] - 0s 114us/sample - loss: 3745.7816
Epoch 4/100
700/700 [==============================] - 0s 72us/sample - loss: 3692.0084
Epoch 5/100
700/700 [==============================] - 0s 46us/sample - loss: 3682.8828
Epoch 6/100
700/700 [==============================] - 0s 49us/sample - loss: 3681.0174
Epoch 7/100
700/700 [==============================] - 0s 51us/sample - loss: 3680.8077
Epoch 8/100
700/700 [==============================] - 0s 50us/sample - loss: 3680.8161
Epoch 9/100
700/700 [==============================] - 0s 66us/sample - loss: 3684.2070
Epoch 10/100
700/700 [==============================] - 0s 68us/sample - loss: 3680.6855
Epoch 11/100
700/700 [==============================] - 0s 51us/sample - loss: 3682.2814
Epoch 12/100
700/700 [======

700/700 [==============================] - 0s 40us/sample - loss: 3682.1663
Epoch 94/100
700/700 [==============================] - 0s 48us/sample - loss: 3682.6191
Epoch 95/100
700/700 [==============================] - 0s 50us/sample - loss: 3680.1466
Epoch 96/100
700/700 [==============================] - 0s 13us/sample - loss: 3680.8182
Epoch 97/100
700/700 [==============================] - 0s 27us/sample - loss: 3682.0528
Epoch 98/100
700/700 [==============================] - 0s 30us/sample - loss: 3681.0594
Epoch 99/100
700/700 [==============================] - 0s 34us/sample - loss: 3679.5063
Epoch 100/100
700/700 [==============================] - 0s 43us/sample - loss: 3684.4124


In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.get_weights()

[array([[ 0.7967203],
        [-0.5240092],
        [-0.2242379],
        [34.175694 ]], dtype=float32), array([33.835545], dtype=float32)]

## Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [47]:
iris_df = pd.read_csv("Iris.csv")

In [48]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

X2 = iris_df.drop(columns= ["Species"] , axis = 1)
y2 = iris_df["Species"]

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [54]:
from sklearn.preprocessing import LabelEncoder

In [77]:
y2_processed = pd.DataFrame(y2).apply(LabelEncoder().fit_transform)


In [78]:
y2_test_cat = tf.keras.utils.to_categorical(y2_processed, num_classes=3)


In [79]:
y2_test_cat

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

### Divide the dataset into Training and test (70:30)

In [80]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2_test_cat, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [81]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


### Fitting the model and predicting 

In [82]:

model.fit(X2_train.values, y2_train, validation_split=0.2,  epochs=30)
         
#model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=trainX.shape[0])

Train on 84 samples, validate on 21 samples
Epoch 1/30
84/84 [==============================] - 1s 11ms/sample - loss: 1.1191 - acc: 0.4286 - val_loss: 2.2693 - val_acc: 0.2857
Epoch 2/30
84/84 [==============================] - 0s 168us/sample - loss: 0.9986 - acc: 0.5119 - val_loss: 2.1055 - val_acc: 0.2857
Epoch 3/30
84/84 [==============================] - 0s 236us/sample - loss: 0.9301 - acc: 0.7262 - val_loss: 2.1361 - val_acc: 0.2857
Epoch 4/30
84/84 [==============================] - 0s 190us/sample - loss: 0.8902 - acc: 0.7738 - val_loss: 2.1451 - val_acc: 0.2857
Epoch 5/30
84/84 [==============================] - 0s 525us/sample - loss: 0.8564 - acc: 0.7857 - val_loss: 2.1494 - val_acc: 0.2857
Epoch 6/30
84/84 [==============================] - 0s 412us/sample - loss: 0.8318 - acc: 0.7738 - val_loss: 2.1376 - val_acc: 0.2857
Epoch 7/30
84/84 [==============================] - 0s 333us/sample - loss: 0.7872 - acc: 0.7738 - val_loss: 2.1177 - val_acc: 0.2857
Epoch 8/30
84/84 [=

In [83]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch multiple                  20        
_________________________________________________________________
dense_10 (Dense)             multiple                  60        
_________________________________________________________________
dense_11 (Dense)             multiple                  88        
_________________________________________________________________
dense_12 (Dense)             multiple                  27        
Total params: 195
Trainable params: 185
Non-trainable params: 10
_________________________________________________________________


### Report Accuracy of the predicted values

In [86]:
y2_predict = model.predict(X2_test)
y2_predict

array([[1.24444067e-03, 9.94611159e-03, 9.88809407e-01],
       [3.18221785e-02, 5.96879460e-02, 9.08489823e-01],
       [4.21570688e-01, 1.49250388e-01, 4.29178894e-01],
       [4.58060019e-02, 7.51945525e-02, 8.78999412e-01],
       [1.34603735e-02, 4.89523076e-02, 9.37587321e-01],
       [2.36088932e-01, 4.13952947e-01, 3.49958092e-01],
       [1.26023982e-02, 2.97935270e-02, 9.57604110e-01],
       [1.23623610e-02, 8.21177065e-02, 9.05519903e-01],
       [6.38292730e-01, 1.16500661e-01, 2.45206654e-01],
       [3.02318316e-02, 4.34067734e-02, 9.26361382e-01],
       [6.37159683e-03, 3.40447873e-02, 9.59583640e-01],
       [2.21986305e-02, 4.13643532e-02, 9.36436951e-01],
       [5.68345070e-01, 5.59843145e-02, 3.75670671e-01],
       [2.02655885e-03, 1.04981735e-02, 9.87475216e-01],
       [3.79929513e-01, 2.01386318e-01, 4.18684214e-01],
       [3.29209752e-02, 5.26052602e-02, 9.14473772e-01],
       [1.28042453e-03, 3.41628790e-02, 9.64556694e-01],
       [1.24687003e-03, 1.22475

In [88]:
model.evaluate(X2_test, y2_test)

45/45 [==============================] - 0s 9ms/sample - loss: 1.2419 - acc: 0.4889


[1.2418707370758058, 0.4888889]